In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta
from tradingview_ta import *

In [9]:
symbol="DOCO"
screener="turkey"
exchange="BIST"

def get_one_analysis(symbol, screener, exchange, interval) -> tradingview_ta.Analysis:
    symbol_analysis = TA_Handler(
        symbol=symbol,
        screener=screener,
        exchange=exchange,
        interval=Interval.INTERVAL_1_HOUR
    )
    return symbol_analysis.get_analysis()

sasa_exp = get_one_analysis(symbol, screener, exchange, Interval.INTERVAL_1_MINUTE)
sasa_exp.summary

{'RECOMMENDATION': 'SELL', 'BUY': 6, 'SELL': 11, 'NEUTRAL': 9}

In [4]:
df = pd.read_csv('./symbol_data/ana_market.csv', index_col='Order')
bist_symbols = df['Code'].to_list()
bist_symbols = [f'bist:{i.lower()}' for i in bist_symbols]

# for initial position -1, then we chack if condition satisfies convert 1, if not 0
bist_symbols_dict = [(i.upper(),-1) for i in bist_symbols] 
bist_symbols_dict = dict(bist_symbols_dict)
# return 'bist:sasa':-1

In [5]:
ind = sasa_exp.indicators

In [12]:
# EMA5 >= SMA10, MACD.macd > MACD.signal, Ichimoku.BLine < close, ADX+DI >= ADX-DI, SMA10 < close, Stoch.K > Stoch.D 
def ind_st_buy_status(stock_ind: dict, symbol: str):
    
    current_price = get_one_analysis(symbol.split(':')[1], screener, exchange, Interval.INTERVAL_1_MINUTE)
    current_price = current_price.indicators['close']
    
    pattern1 = stock_ind['EMA5'] >= stock_ind['SMA10']
    pattern2 = stock_ind['MACD.macd'] > stock_ind['MACD.signal']
    pattern3 = stock_ind['Ichimoku.BLine'] < current_price
    pattern4 = stock_ind['ADX+DI'] >= stock_ind['ADX-DI']
    pattern5 = stock_ind['SMA10'] < current_price
    pattern6 = stock_ind['Stoch.K'] > stock_ind['Stoch.D']

    if(pattern1 and pattern2 and pattern3 and pattern4 and pattern5 and pattern6):
        #print(f'Buy the stock {symbol}')
        return symbol.split(':')[1]
    else:
        return None

In [ ]:
def get_st_result_list(screener, interval, symbols) -> list:
    result_list = []
    analysis = get_multiple_analysis(
        screener=screener, 
        interval=interval, 
        symbols=symbols
        )
    for k in analysis:
        dict_ind = analysis[k].indicators
        temp_symbol = ind_st_buy_status(dict_ind, k)
        if temp_symbol != None:
            result_list.append(temp_symbol)
    return result_list

buy_list = get_st_result_list(screener, Interval.INTERVAL_1_DAY, bist_symbols)

In [17]:
def ind_st_buy_signal(stock_ind: dict, symbol_pair: dict, symbol: str):

    pattern1 = stock_ind['EMA5'] >= stock_ind['SMA10']
    pattern2 = stock_ind['MACD.macd'] > stock_ind['MACD.signal']
    pattern3 = stock_ind['Ichimoku.BLine'] < stock_ind['close']
    pattern4 = stock_ind['ADX+DI'] >= stock_ind['ADX-DI']
    pattern5 = stock_ind['SMA10'] < stock_ind['close']
    pattern6 = stock_ind['Stoch.K'] > stock_ind['Stoch.D']

    all_pattern = (pattern1 and pattern2 and pattern3 and pattern4 and pattern5 and pattern6)

    if all_pattern and symbol_pair[symbol] == 0:
        print(f'BUY SIGNAL FOR {symbol} at {stock_ind["close"]}')
        symbol_pair[symbol] = 1
    elif all_pattern and symbol_pair[symbol] == -1:
        symbol_pair[symbol] = 1
    elif not all_pattern and symbol_pair[symbol] == 1:
        print(f'SELL SIGNAL FOR {symbol} at {stock_ind["close"]}')
        symbol_pair[symbol] = 0
    else:
        symbol_pair[symbol] = 0

In [18]:
import time

def listen_live_signal(screener, interval, symbols, sleep_time) -> None:
    while True:
        analysis = get_multiple_analysis(
            screener=screener, 
            interval=interval, 
            symbols=symbols
            )
        for k in analysis:
            dict_ind = analysis[k].indicators
            ind_st_buy_signal(dict_ind, bist_symbols_dict, k)
        time.sleep(60*10)    

In [ ]:
listen_live_signal(screener, Interval.INTERVAL_1_HOUR, bist_symbols, 60*10)